In [141]:
import requests
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [99]:
df = pd.DataFrame()

In [100]:
def get_shopee_seller_by_keyword_one_request(key, limit = 50, offset = 0) :
    data = requests.get(f"https://shopee.vn/api/v4/search/search_user?keyword={key}&limit={limit}&offset={offset}&page=search_user&with_search_cover=true").json()
    if data['data'] is not None:
        return data['data']['users']
    else:
        return None

In [101]:
def filter_attribute_shop(shop) :
    shop_filtered = dict()
    attr = ['name', 'country', 'follower_count','products', 'shopid', 'ctime', 'rating_normal', 'rating_bad', 'rating_good', 'item_count', 'rating_star', 'shop_location', 'response_time', 'response_rate']
    for i in attr :
        shop_filtered[i] = shop[i]
    return shop_filtered

In [102]:
def get_seller_more_detail_one_request(seller) :
    data = requests.get(f'https://shopee.vn/api/v4/shop/get_shop_detail?sort_sold_out=0&username={seller["username"]}').json()
    return data['data']

In [103]:
def convert_to_seller_more_detail(sellers) :
    sellers_more_detail = []
    for seller in sellers :
        seller_more = get_seller_more_detail_one_request(seller) 
        seller_more['products'] = seller['products']
        sellers_more_detail.extend([seller_more])
    return sellers_more_detail

In [104]:
def get_shopee_seller_by_keyword(key) :
    key = key.replace(' ', '%20')
    data = requests.get(f"https://shopee.vn/api/v4/search/search_user?keyword={key}&limit=6&offset=0&page=search_user&with_search_cover=true").json()
    total = data['data']['total_count']
    limit = 100
    max_request = total//limit + 1
    sellers = []
    bar = tqdm(range(max_request), desc="Running")
    for i in bar:
        seller = get_shopee_seller_by_keyword_one_request(key,limit=limit, offset=limit*i)
        if seller is not None:
            seller_converted = convert_to_seller_more_detail(seller)
            bar.set_description(f'{i}_{len(seller_converted)}')
            sellers.extend([filter_attribute_shop(s) for s in seller_converted])
    return sellers

In [186]:
while 1:
    key = 'áo khoác'.replace(' ', '%20')
    limit = 100
    offset = np.random.choice(500, 1)[0]
    seller = get_shopee_seller_by_keyword_one_request(key,limit=limit, offset=offset)
    if seller is not None:
        seller_converted = convert_to_seller_more_detail(seller)
        seller = [filter_attribute_shop(s) for s in seller_converted]
        
    seller = pd.DataFrame(seller)
    df = pd.concat([df, seller], ignore_index=True)
    df = df.drop_duplicates(subset=['shopid']).reset_index(drop=True)
    print(len(df))
    if len(df) > 3300:
        break

660
660


KeyboardInterrupt: 

In [187]:
df.drop_duplicates(subset=['shopid'], keep='first', inplace=True)
df.to_csv('seller.csv', index=False)

In [188]:
sellers = df.to_dict('records')

In [189]:
def get_item_from_shop_one_request(key, shopid, limit=50, offset=0) :
    data = requests.get(f"https://shopee.vn/api/v4/recommend/recommend?bundle=shop_page_homepage&item_card=2&keyword={key}&limit={limit}&offset={offset}&section=shop_page_homepage_rfu_sec&shopid={shopid}&upstream=search").json()
    return data['data']['sections'][0]['data']['item']

In [190]:
def normalize(sentence) :
    sentence = unicodedata.normalize("NFC", sentence)
    sentence = sentence.lower()
    sentence = re.sub(r"\s+", ' ', sentence)
    return sentence

In [191]:
import unicodedata
def filter_product_by_key_with_name(product , key) :
    key = unicodedata.normalize("NFC", key)
    name_nor = normalize(unicodedata.normalize("NFC", product['name']))
    if key in name_nor or 'bommer' in name_nor:
        return True
    return False

In [192]:
def get_item_from_shop(key, shop) :
    key_replaced = key.replace(' ', '%20')
    total = shop['products']
    max_request = total//50 + 1
    products = []
    for i in range(max_request) :
        product = get_item_from_shop_one_request(key_replaced, shop['shopid'], limit=50,offset= 50*i)
        if product is not None:
            p = [p for p in product if filter_product_by_key_with_name(p, key)]
            products.extend(p)
    return products 

In [193]:
def filter_attribute_product(item) :
    product = dict()
    attr = ['itemid', 'shopid', 'name', 'stock', 'ctime', 'sold', 'historical_sold', 'brand', 'catid', 'price', 'price_min', 'price_max', 'tier_variations']
    for i in attr :
        product[i] = item[i]
    return product

In [194]:
def shopee_thread(sellers, key):
    list_products = []
    for i in tqdm(range(len(sellers))):
        list_product = [filter_attribute_product(item) for item in get_item_from_shop(key, sellers[i])]
        list_products += list_product
    return list_products

In [195]:
products = shopee_thread(sellers, 'áo khoác')

100%|█████████████████████████████████████████████████████████████████████████████████| 660/660 [36:15<00:00,  3.30s/it]


In [211]:
x = pd.DataFrame(products)
cols = x.columns[:-1]
x.drop_duplicates(subset=cols, inplace=True, keep='first')
x.to_csv('product.csv', index=False)

### TIKI >>>>>>>>>>>>>>>>>>

In [ ]:
header = {
    'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0'}

In [ ]:
def get_list_product_one_request_tiki(key, limit=40, page=1) :
  data = requests.get(f"https://tiki.vn/api/v2/products?limit={limit}&include=advertisement&aggregations=2&q={key}&page={page}", headers=header).json()
  return data['data']

In [ ]:
def filter_attribute_product_tiki(product) :
  product_filtered = dict()
  attr = ['id', 'name', 'availability', 'seller_id', 'seller_name', 'brand_id', 'price', 'original_price', 'discount', 'rating_average']
  for i in attr :
    product_filtered[i] = product[i]
  product_filtered['sold'] = product['quantity_sold']['value']
  return product_filtered

In [ ]:
def get_list_product_tiki(key) :
  key_replaced = key.replace(' ', '+')
  data = requests.get(f"https://tiki.vn/api/v2/products?limit=40&include=advertisement&aggregations=2&q={key_replaced}", headers=header).json()
  total_page = data['paging']['last_page']
  print(total_page)
  products = []
  for i in range(1,total_page+1) :
    product = get_list_product_one_request_tiki(key_replaced, limit=40, page=i)
    products.extend([filter_attribute_product_tiki(p) for p in product])
    break
  return products

In [ ]:
products = get_list_product_tiki('áo khoác')

In [ ]:
products[0]

In [ ]:
def get_info_shop_one_request_tiki(shopid) :
  data = requests.get(f"https://tiki.vn/api/shopping/v2/widgets/seller?seller_id={shopid}&platform=desktop").json()
  